# Data prep

## In this notebook we will load the songs from the zip file, and perform transformations to prepare the data for two-tower training
Steps
1. Extract from the zip file
2. Upload to BQ
3. Enrich features for the playlist songs
4. Cross-join songs with features (expected rows = n_songs x n_playlists)
5. Remove after-the-fact (later position songs) from the newly generated samples
6. Create a clean train table, and flatten structs or use arrays

In [1]:
# Set your variables for your project, region, and dataset name
SOURCE_BUCKET = 'spotify-million-playlist-dataset'
PROJECT_ID = 'wortz-project-352116'
REGION = 'us-central1'
bq_dataset = 'spotify_e2e_test'

import time
from google.cloud import bigquery

bigquery_client = bigquery.Client(project=PROJECT_ID, location='US')

## Now enrich the playlist songs with the new features

`audio_features` - created from prior notebook via Spotify API

+

`artist_features` - created from prior notebook via Spotify API

These are additional tables where features were added in the beginning of the notebook

In [29]:
%%time
enrich_query = f"""CREATE OR REPLACE TABLE
  `{PROJECT_ID}.{bq_dataset}.enriched_data` AS (
  WITH tf as (SELECT distinct * from `{PROJECT_ID}.{bq_dataset}.audio_features`),
       af as (SELECT distinct * from `{PROJECT_ID}.{bq_dataset}.artist_features`) 
  
    SELECT
    a.* except(tracks),
      ARRAY(
    SELECT
      AS STRUCT CAST(track.pos AS int64) AS pos_can,
      case when track.artist_name = '' then 'NONE' else track.artist_name end AS artist_name_can,
      case when track.track_uri = '' then 'NONE' else track.track_uri  end AS track_uri_can,
      case when track.album_uri = '' then 'NONE' else track.album_uri  end AS album_uri_can,
      case when track.artist_uri = '' then 'NONE' else track.artist_uri  end AS artist_uri_can,
      case when track.track_name = '' then 'NONE' else track.track_name end AS track_name_can,
      CAST(track.duration_ms AS float64) / 1.0 AS duration_ms_can,
      case when track.album_name = '' then 'NONE' else track.album_name end AS album_name_can,
      CAST(IFNULL(tf.track_pop, 0.0) as float64) / 1.0 AS track_pop_can,
      CAST(IFNULL(af.artist_pop, 0.0) as float64) / 1.0  AS artist_pop_can,
      CAST(IFNULL(af.genres, "['NONE']") as string)  AS artist_genres_can,
      CAST(IFNULL(af.followers, 0.0) as float64) / 1.0 AS artist_followers_can
    FROM
      UNNEST(tracks) as track
    INNER JOIN
     tf --track features
    ON
      (track.track_uri = tf.track_uri)
    INNER JOIN
      af
      ON
      (track.artist_uri = af.artist_uri)
      ) AS tracks
  FROM 
  `{PROJECT_ID}.{bq_dataset}.playlists_nested` as a)"""

bigquery_client.query(enrich_query).result()

CPU times: user 65.5 ms, sys: 7.5 ms, total: 73 ms
Wall time: 44.2 s


## Cross join + get rid of after-the-fact `pos` data in playlist

cross_join_songxplaylist_struct_query

`hybrid-vertex.spotify_train_3.ordered_position_training`

We create a data structure that creates unique song-playlist combos (every possible via cross-join). There is also a portion of pulling the last song in the playlist as the "seed track"
________
### Note on the approach

Semantic matching requires pairs, triplets (tuples generally) of co-occurrences between pairs. This is a very broad definition, and with this newer approach many new use cases are being explored. A simple example are finding pairs of user queries and purchases. The training example pair are: (the features we know from the user query, the features we know on the product they ultimately purchased).

There are other approaches where triples are considered, and there are advanced techniques on negative sampling, finding “bad” examples of query, product pairs, which we will not cover here.

Note there are other sampling techniques we highlight below (different artist/album)

The chosen task was predicting the next song on a playlist, given the playlist existing order. The approach taken was to create pairs for all children songs and their parent playlists. We did leveraging BigQuery’s `UNNEST` and `CROSS JOIN`. 

We also had rich features for playlists, albums and songs in another table that was later used to enrich post `CROSS JOIN`. This was done to optimize the computation since the cross-joining is expensive and it was subsequently much quicker to enrich after this step.

Now that we completed this step, we had all combinations of child song, playlist pairs. The song was the candidate label but the playlist still contained the candidate label and all songs after. Additional criteria was added to remove the candidate song and all songs that occur after the candidate in the playlist. For the sake of performance we also only considered the last 5 played songs. Other sampling configurations are available in the example notebook as well (only predicting when there are album and artist switches).

What this results in is a training dataset that has all possible child song candidates joined with the full playlist data, and the playlist data is properly censored as to only contain songs up to before the candidate song.

![](img/semantic-pair.png)

In [30]:
%%time
cross_join_query = f"""
  CREATE OR REPLACE TABLE
  `{PROJECT_ID}.{bq_dataset}.ordered_position_training` AS (
  WITH
    -- get every combination of song and its parent playlist
    unnest_cross AS(
    SELECT
      b.*,
      CONCAT(b.pid,"-",track.pos_can) AS pid_pos_id,
      CAST(track.pos_can AS int64) AS pos_can,
      IFNULL(track.artist_name_can, "NONE") as artist_name_can ,
      track.track_uri_can ,
      track.album_uri_can,
      IFNULL(track.track_name_can, "NONE") as track_name_can ,
      track.artist_uri_can ,
      CAST(track.duration_ms_can AS float64) AS duration_ms_can,
      track.album_name_can ,
      track.track_pop_can ,
      track.artist_pop_can,
      track.artist_genres_can as artist_genres_can ,
      track.artist_followers_can 
    FROM (
      SELECT
        * EXCEPT(duration_ms)
      FROM
        `{PROJECT_ID}.{bq_dataset}.enriched_data`) AS b
    CROSS JOIN
      UNNEST(tracks) AS track)
  SELECT
    a.* EXCEPT(tracks,
      num_tracks,
      num_artists,
      num_albums,
      num_followers,
      num_edits),
    ARRAY(
    SELECT
      AS STRUCT CAST(track.pos_can AS int64) AS pos_pl,
      track.artist_name_can AS artist_name_pl,
      track.track_uri_can AS track_uri_pl,
      track.track_name_can AS track_name_pl,
      track.album_uri_can AS album_uri_pl,
      track.artist_uri_can AS artist_uri_pl,
      CAST(track.duration_ms_can AS float64) AS duration_ms_pl,
      track.album_name_can AS album_name_pl,
      track.track_pop_can AS track_pop_pl,
      track.artist_pop_can AS artist_pop_pl,
      track.artist_genres_can AS artist_genres_pl,
      track.artist_followers_can AS artist_followers_pl,
    FROM
      UNNEST(tracks) AS track
    WHERE
      CAST(track.pos_can AS int64) < a.pos_can ORDER BY CAST(track.pos_can AS int64)) AS seed_playlist_tracks
  FROM
    unnest_cross AS a -- with statement
    )
    """

bigquery_client.query(cross_join_query).result()

CPU times: user 46.1 ms, sys: 9.64 ms, total: 55.7 ms
Wall time: 2min 45s


## Update the playlist metadata with the new samples created above

Add audio features from the tracks

Get new metadata for the tracks now that there are updated track counts, durations, etc...

`hybrid-vertex.spotify_train_3.train` will be produced


In [31]:
%%time
get_new_metadata_query = f"""
create or replace table `{PROJECT_ID}.{bq_dataset}.train` as (
WITH
  playlist_features_clean AS (
  SELECT
    pid_pos_id,
    SUM(trx.duration_ms_pl) / 1.0 AS duration_ms_seed_pl,
    COUNT(1) / 1.0 AS n_songs_pl,
    COUNT(DISTINCT trx.artist_name_pl) / 1.0 AS num_artists_pl,
    COUNT(DISTINCT trx.album_uri_pl) /1.0 AS num_albums_pl,
  FROM
    `{PROJECT_ID}.{bq_dataset}.ordered_position_training`,
    UNNEST(seed_playlist_tracks) AS trx
  GROUP BY
    pid_pos_id)
    
SELECT
  a.* except(artist_genres_can, track_pop_can, artist_pop_can, artist_followers_can),
  b.* except(pid_pos_id),
  a.artist_genres_can,
  IFNULL(a.track_pop_can, 0.0) / 1.0 as  track_pop_can, 
  IFNULL(a.artist_pop_can, 0.0) / 1.0 as artist_pop_can,
  IFNULL(a.artist_followers_can, 0.0) / 1.0 as artist_followers_can,

  
FROM
  `{PROJECT_ID}.{bq_dataset}.ordered_position_training` a
INNER JOIN
  playlist_features_clean b
ON
  a.pid_pos_id = b.pid_pos_id )
  """

bigquery_client.query(get_new_metadata_query).result()

CPU times: user 81.9 ms, sys: 12.5 ms, total: 94.4 ms
Wall time: 2min 26s


In [32]:
%%time
### Get candidates

get_unique_candidates = f"""
create or replace table `{PROJECT_ID}.{bq_dataset}.candidates` as (
WITH
af as (SELECT DISTINCT * FROM `{PROJECT_ID}.{bq_dataset}.audio_features`)

SELECT DISTINCT
    track_uri_can,
    track_name_can,
    artist_uri_can,
    artist_name_can,
    album_uri_can,
    album_name_can,
    duration_ms_can,
    track_pop_can,
    artist_pop_can,
    artist_genres_can,
    artist_followers_can,
    
    IFNULL(af.danceability, 0.) as track_danceability_can,
    IFNULL(af.energy, 0.) as track_energy_can,
    IFNULL(af.key, 0.) as track_key_can,
    IFNULL(af.loudness, 0.) as track_loudness_can,
    IFNULL(af.mode, 0) as track_mode_can,
    IFNULL(af.speechiness, 0.) as track_speechiness_can,
    IFNULL(af.acousticness, 0.) as track_acousticness_can,
    IFNULL(af.instrumentalness, 0.) as track_instrumentalness_can,
    IFNULL(af.liveness, 0.) as track_liveness_can,
    IFNULL(af.valence, 0.) as track_valence_can,
    IFNULL(af.tempo, 0.) as track_tempo_can,
    IFNULL(af.time_signature, 0) as time_signature_can,
FROM
  `{PROJECT_ID}.{bq_dataset}.train` a
   inner join af on af.track_uri = a.track_uri_can
  )
  """

bigquery_client.query(get_unique_candidates).result()

CPU times: user 16.1 ms, sys: 0 ns, total: 16.1 ms
Wall time: 15.4 s


## For TFRecords
Get rid of structs by creating new table with arrays from playlist_seed


# Only selecting last 5 songs

song_history is settable but it will impact `MAX_PLAYLIST_LENGTH` in `src/two_tower.py`

In [33]:
TRACK_HISTORY=5 # length of playlist tracks to consider

In [34]:
%%time

train_flatten_query = f"""
create or replace table `{PROJECT_ID}.{bq_dataset}.train_flatten_pre_split_a` as (
WITH audio as (SELECT DISTINCT * FROM `{PROJECT_ID}.{bq_dataset}.audio_features`)
SELECT 
    pid,
    IFNULL(a.name, "") as pl_name_src,
    collaborative as pl_collaborative_src,
    duration_ms_seed_pl as pl_duration_ms_new,
    n_songs_pl as num_pl_songs_new, 
    num_artists_pl as num_pl_artists_new,
    num_albums_pl as num_pl_albums_new,
    track_uri_can,
    track_name_can,
    artist_uri_can,
    artist_name_can,
    album_uri_can,
    album_name_can,
    duration_ms_can,
    track_pop_can,
    artist_pop_can,
    artist_genres_can,
    artist_followers_can,
    IFNULL(audio.danceability, 0.0) as track_danceability_can,
    IFNULL(audio.energy, 0.0) as track_energy_can,
    IFNULL(audio.key, 0.0) as track_key_can,
    IFNULL(audio.loudness, 0.0) as track_loudness_can,
    IFNULL(audio.mode, 0) as track_mode_can,
    IFNULL(audio.acousticness, 0.0) as track_acousticness_can,
    IFNULL(audio.instrumentalness, 0.0) as track_instrumentalness_can,
    IFNULL(audio.liveness, 0.0) as track_liveness_can,
    IFNULL(audio.speechiness, 0.0) as track_speechiness_can,
    IFNULL(audio.valence, 0.0) as track_valence_can,
    IFNULL(audio.tempo, 0.0) as track_tempo_can,
    IFNULL(audio.time_signature, 0) as track_time_signature_can,
    
    ARRAY(select t.artist_name_pl from UNNEST(seed_playlist_tracks) t 
        where pos_pl >= pos_can - {TRACK_HISTORY}) as artist_name_pl, 
    ARRAY(select t.artist_uri_pl from UNNEST(seed_playlist_tracks) t 
        where pos_pl >= pos_can - {TRACK_HISTORY}) as artist_uri_pl, 
    ARRAY(select t.track_uri_pl from UNNEST(seed_playlist_tracks) t 
        where pos_pl >= pos_can - {TRACK_HISTORY}) as track_uri_pl,
    ARRAY(select t.track_name_pl from UNNEST(seed_playlist_tracks) t 
        where pos_pl >= pos_can - {TRACK_HISTORY}) as track_name_pl,
    ARRAY(select t.duration_ms_pl from UNNEST(seed_playlist_tracks) t 
        where pos_pl >= pos_can - {TRACK_HISTORY}) as duration_ms_songs_pl, 
    ARRAY(select t.album_name_pl from UNNEST(seed_playlist_tracks) t 
        where pos_pl >= pos_can - {TRACK_HISTORY}) as album_name_pl,
    ARRAY(select t.album_uri_pl from UNNEST(seed_playlist_tracks) t 
        where pos_pl >= pos_can - {TRACK_HISTORY}) as album_uri_pl,
    ARRAY(select cast(t.artist_pop_pl as FLOAT64) from UNNEST(seed_playlist_tracks) t 
        where pos_pl >= pos_can - {TRACK_HISTORY}) as artist_pop_pl,
    ARRAY(select t.artist_followers_pl from UNNEST(seed_playlist_tracks) t 
        where pos_pl >= pos_can - {TRACK_HISTORY}) as artists_followers_pl,
    ARRAY(select t.track_pop_pl from UNNEST(seed_playlist_tracks) t 
        where pos_pl >= pos_can - {TRACK_HISTORY}) as track_pop_pl, 
    ARRAY(select t.artist_genres_pl from UNNEST(seed_playlist_tracks) t 
        where pos_pl >= pos_can - {TRACK_HISTORY}) as artist_genres_pl
    
    from `{PROJECT_ID}.{bq_dataset}.train` a
    INNER JOIN 
    audio on audio.track_uri = a.track_uri_can
)
"""

bigquery_client.query(train_flatten_query).result()

CPU times: user 29.8 ms, sys: 94 µs, total: 29.9 ms
Wall time: 56.4 s


#### Append the audio features

In [35]:
%%time

train_flatten_query = f"""
create or replace table `{PROJECT_ID}.{bq_dataset}.train_flatten_pre_split_b` as (
WITH audio as (SELECT DISTINCT * FROM `{PROJECT_ID}.{bq_dataset}.audio_features`)
SELECT 
    a.*,
    ARRAY(select IFNULL(audio.danceability, 0.0) from UNNEST(a.track_uri_pl) t, audio where audio.track_uri = t) as track_danceability_pl,
    ARRAY(select IFNULL(audio.energy, 0.0) from UNNEST(a.track_uri_pl) t, audio where audio.track_uri = t) as track_energy_pl,
    ARRAY(select IFNULL(audio.key, 0.0) from UNNEST(a.track_uri_pl) t, audio where audio.track_uri = t) as track_key_pl,
    ARRAY(select IFNULL(audio.loudness, 0.0) from UNNEST(a.track_uri_pl) t, audio where audio.track_uri = t) as track_loudness_pl,
    ARRAY(select IFNULL(audio.mode, 0) from UNNEST(a.track_uri_pl) t, audio where audio.track_uri = t) as track_mode_pl,
    ARRAY(select IFNULL(audio.acousticness, 0.0) from UNNEST(a.track_uri_pl) t, audio where audio.track_uri = t) as track_acousticness_pl,
    ARRAY(select IFNULL(audio.instrumentalness, 0.0) from UNNEST(a.track_uri_pl) t, audio where audio.track_uri = t) as track_instrumentalness_pl,
    ARRAY(select IFNULL(audio.liveness, 0.0) from UNNEST(a.track_uri_pl) t, audio where audio.track_uri = t) as track_liveness_pl,
    ARRAY(select IFNULL(audio.valence, 0.0) from UNNEST(a.track_uri_pl) t, audio where audio.track_uri = t) as track_valence_pl,
    ARRAY(select IFNULL(audio.tempo, 0.0) from UNNEST(a.track_uri_pl) t, audio where audio.track_uri = t) as track_tempo_pl,
    ARRAY(select IFNULL(audio.time_signature, 0) from UNNEST(a.track_uri_pl) t, audio where audio.track_uri = t) as track_time_signature_pl,
    ARRAY(select IFNULL(audio.speechiness, 0) from UNNEST(a.track_uri_pl) t, audio where audio.track_uri = t) as track_speechiness_pl
    from `{PROJECT_ID}.{bq_dataset}.train_flatten_pre_split_a` a
    WHERE
     ARRAY_LENGTH(a.track_uri_pl) = {TRACK_HISTORY}) --limiting here for performance
"""

bigquery_client.query(train_flatten_query).result()

CPU times: user 315 ms, sys: 43 ms, total: 358 ms
Wall time: 26min 5s


## Important for validation strategy
Different playlist ids were selected for validation to prevent cross-contamination with the sampling approach.

In [36]:
%%time
VALIDATION_P = 0.01

validation_creation = f"""
CREATE OR REPLACE TABLE
  `{PROJECT_ID}.{bq_dataset}.train_flatten_valid_last_{TRACK_HISTORY}` AS (
    SELECT * 
  FROM
    `{PROJECT_ID}.{bq_dataset}.train_flatten_pre_split_b` where MOD(pid, 100) = 0
    AND ARRAY_LENGTH(track_uri_pl) = {TRACK_HISTORY})""" #complete examples only

bigquery_client.query(validation_creation).result()

CPU times: user 12.3 ms, sys: 4.42 ms, total: 16.7 ms
Wall time: 17.4 s


In [37]:
%%time
VALIDATION_P = 0.01

validation_creation = f"""
CREATE OR REPLACE TABLE
  `{PROJECT_ID}.{bq_dataset}.train_flatten_last_{TRACK_HISTORY}` AS (
    SELECT * 
  FROM
    `{PROJECT_ID}.{bq_dataset}.train_flatten_pre_split_b` where MOD(pid, 100) != 0
    AND ARRAY_LENGTH(track_uri_pl) = {TRACK_HISTORY})""" #complete examples only"""

bigquery_client.query(validation_creation).result()

CPU times: user 25.1 ms, sys: 0 ns, total: 25.1 ms
Wall time: 32.8 s


## Done - you can move on to the [next notebook](02-tfrecord-beam-pipeline.ipynb) 

Your data should look like this:
    
![](img/train-dataset-metadata.png)